In [ ]:
import requests
import pandas as pd
import itertools

In [ ]:
# 발급 받은 API 키
YOUR_API_KEY = '640b6edf35bb47f280aac61745395c9e'
# 국회의원 발의법률안 데이터 아이디
data_id = 'nzmimeepazxkubdpn'

In [ ]:
url = f"https://open.assembly.go.kr/portal/openapi/{data_id}?KEY={YOUR_API_KEY}&AGE=21&Type=json"
response = requests.get(url).json()

In [ ]:
# 한 번에 100개씩 밖에 데이터를 못가져 온다...
len(response[data_id][1]['row'])

In [ ]:
max_page = response[data_id][0]['head'][0]['list_total_count']
max_page

In [ ]:
all_laws = []
for page_num in range(1, int(max_page/1000+2)):
    p_size= page_num*1000
    p_index = page_num
    url = f"https://open.assembly.go.kr/portal/openapi/{data_id}?KEY={YOUR_API_KEY}&AGE=21&Type=json&pIndex={page_num}&pSize=1000"
    laws = requests.get(url).json()[data_id][1]['row']
    all_laws.extend(laws)
all_laws = pd.DataFrame(all_laws)
all_laws.head()

In [ ]:
# 혹시 날아갈까, 데이터를 파일로 저장하자.
all_laws.to_csv("all_laws.csv",index=False)

In [ ]:
# 이번에는 국회의원들의 프로필을 털어보자.
data2_id = 'nwvrqwxyaytdsfvhu'
url =f"https://open.assembly.go.kr/portal/openapi/{data2_id}?KEY={YOUR_API_KEY}&Type=json&UNIT_CD=100020&pSize=1000"
senators = requests.get(url).json()[data2_id][1]['row']
senators = pd.DataFrame(senators)
senators

In [ ]:
# 혹시 날아갈까, 데이터를 파일로 저장하자.
senators.to_csv("senators.csv",index=False)

In [ ]:
# 아래는 같은 법률로 발의된 국회의원들의 조합을 찾아서, 그래프를 그리기 위한 데이터를 만드는 과정이다.
all_laws['PUBL_PROPOSER'] = all_laws['PUBL_PROPOSER']+','+all_laws['RST_PROPOSER']
all_laws['senator'] = all_laws['PUBL_PROPOSER'].str.split(',')
all_laws = all_laws[all_laws['senator'].notnull()]
pairs = all_laws['senator'].apply(lambda x: x if x is None else list(itertools.combinations(x,2))).explode()
pairs = pd.DataFrame(pairs.tolist(), columns=['Source', 'Target'])
pairs = pairs[pairs['Target'].notnull()]

In [ ]:
# edge_list.csv 파일로 저장하자.
pairs.to_csv("edge_list.csv", index=False)

In [ ]:
# 국회의원들은 노드 리스트로 만들어서 저장하자.
node_list = senators[['HG_NM','HG_NM', 'POLY_NM', 'ORIG_NM', 'SEX_GBN_NM']]
node_list.columns = ['Id','Label','Party','Local', 'Gender']
node_list.to_csv("node_list.csv",index=False)

### [Gephi](https://gephi.org/)에서 실습?